In [31]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import os
import glob
import random
import shutil

img_path="data/images/"
split_path = "data/split"

In [30]:
##view of excel sheet
import pandas as pd
excel = pd.read_csv("Data_Entry_2017.csv")
excel.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [4]:
#delete all exisitng folders
shutil.rmtree("images/train/disease")
shutil.rmtree("images/train/No_Finding")
shutil.rmtree("images/test/disease")
shutil.rmtree("images/test/No_Finding")
shutil.rmtree("images/valid/disease")
shutil.rmtree("images/valid/No_Finding")

SyntaxError: unexpected EOF while parsing (<ipython-input-4-b215d390990e>, line 8)

In [ ]:
#splitting the train and test files
import shutil
def movefile(filenamelist_txt,des_folder):
    f = open(filenamelist_txt, "r")
    if not os.path.exists(os.path.join(img_path,des_folder)):
        os.makedirs(os.path.join(img_path,des_folder))
    for x in f:
        x = x.rstrip() #removing \n
        sorce_file_abs = os.path.join(os.getcwd(),os.path.join(img_path,x))
        destination_file_abs = os.path.join(os.getcwd(),os.path.join(os.path.join(img_path,des_folder),x))
        shutil.move(sorce_file_abs, destination_file_abs)
movefile('test_list.txt','test')
movefile('train_val_list.txt','train')

def count_files(directory_path):
    
f = open('train_val_list.txt', "r")
image_files = []
for x in f:
    x = x.rstrip()
    image_files.append(x)
len(image_files)

In [5]:
#splitting data into their respective class
import csv

def create_label_directories(csv_filename, img_path, is_one_v_all=False, one_v_all_label="No Finding"):
    directories = set()
    with open(csv_filename, 'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        next(csvfile) # skip header row
        for row in reader:
            img_filename = str(row[0])
            labels = str(row[1])
            src_file = os.path.join(img_path,img_filename)
            # getting the source file absolute path    
            src_file_abs = os.path.join(os.getcwd(),src_file)
            if not os.path.exists(src_file_abs):
                continue
            for label in labels.split('|'):
                if (is_one_v_all is True) and (label != one_v_all_label):
                    label = "disease"
                label = "_".join(label.split())
                dst_train_dir = os.path.join(img_path,label)
                dst_train_file = os.path.join(dst_train_dir,img_filename)
                dst_valid_dir = os.path.join("images","valid",label)
                # making all the directories
                if not os.path.exists(dst_train_dir):
                    os.makedirs(dst_train_dir)
                    directories.add(label)
                if not os.path.exists(dst_valid_dir):
                    os.makedirs(dst_valid_dir)
                # getting the destination file absolute path
                dst_train_file_abs = os.path.join(os.getcwd(),dst_train_file)
                if not os.path.exists(dst_train_file_abs):
                    os.symlink(src_file_abs, dst_train_file_abs)
    return list(directories)

is_one_v_all = False
one_v_all_label = "No Finding"

directories = create_label_directories("Data_Entry_2017.csv", 'images/train/' , is_one_v_all, one_v_all_label)
directories = create_label_directories("Data_Entry_2017.csv", 'images/test/' , is_one_v_all, one_v_all_label)
print(directories)

['Nodule', 'Consolidation', 'Pneumothorax', 'Pneumonia', 'Emphysema', 'Mass', 'Infiltration', 'Cardiomegaly', 'Fibrosis', 'No_Finding', 'Effusion', 'Hernia', 'Pleural_Thickening', 'Atelectasis', 'Edema']


In [21]:
path, dirs, files = os.walk('images/test/Fibrosis').__next__()
print(path)
file_count = len(files)
print(file_count)

images/test/Fibrosis
435


In [28]:
#downsampling the data
def get_per_label_count(image_folder_path,directories):
    per_label_count = []
    for ii in range(len(directories)):
        for path, dirs, files in os.walk(os.path.join(image_folder_path,directories[ii])):
            file_count = len(files)
            per_label_count.append(file_count)
    return per_label_count
        
def downsample(path, directories, per_label_count):
    label_idx = np.argmin(per_label_count)
    downsample_count = per_label_count[label_idx]
    for ii in range(len(per_label_count)):
        label = directories[ii]
        src_train_dir = os.path.join(path,label)
        all_img_paths = glob.glob(os.path.join(src_train_dir,"*.*"))
        np.random.shuffle(all_img_paths)
        if len(all_img_paths) != downsample_count:
            imgs_to_remove = all_img_paths[downsample_count:]
            for file in imgs_to_remove:
                file_abs = os.path.join(os.getcwd(),file)
                os.remove(file_abs)
    return get_per_label_count(path,directories)

# per_label_count = get_per_label_count(os.path.join(img_path,"train"), directories) 

# print("Downsampling Training set")
# print("Before", per_label_count)
# per_label_count = downsample(os.path.join(img_path,"train"), directories, per_label_count)
# print("After", per_label_count)

per_label_count = get_per_label_count(os.path.join(img_path,"test"), directories)
print("Downsampling Testing set")
print("Before", per_label_count)
per_label_count = downsample(os.path.join(img_path,"test"), directories , per_label_count)
print(per_label_count)

Downsampling Testing set
Before [86, 1815, 2665, 555, 1093, 1748, 6112, 1069, 435, 4658, 86, 1143, 3279, 925]
[86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 925]


In [ ]:
#splitting train data in to train and validation
def split_train_valid_test(directories, per_label_count, valid_pct):
    for ii in range(len(directories)):
        all_img_paths = glob.glob(os.path.join(img_path,"train",directories[ii],"*.*"))
        #shuffling all training images
        np.random.shuffle(all_img_paths)
        label_count = per_label_count[ii]
        valid_count = int(label_count*valid_pct)
        valid_files = all_img_paths[:valid_count]
        all_img_paths[:valid_count] = []
        train_files = all_img_paths
        all_img_paths = []
        for valid_file in valid_files:
            valid_file_abs = os.path.join(os.getcwd(),valid_file)
            shutil.move(valid_file_abs, os.path.join(os.getcwd(),img_path,"valid",directories[ii],os.path.basename(valid_file)))

valid_pct = 0.02      

split_train_valid_test(directories, per_label_count, valid_pct)